### Inference avec une pipeline

In [13]:
from transformers import pipeline

classifier = pipeline("text-classification", model="model")

### Test 1

Le premier texte est une réponse de Chat GPT à la question "Qui a inventé les feux de circulation ?" 

In [14]:
text_1 = "The modern traffic light, as we know it today, was invented by Garrett Morgan, an African-American inventor and entrepreneur. In 1923, Morgan patented an improved version of the traffic signal, which included a third position to stop traffic in all directions to allow pedestrians to cross streets safely. Morgan's invention was a significant contribution to traffic management and road safety."

classifier(text_1)

[{'label': 'CHATGPT', 'score': 0.9999736547470093}]

### Test 2

Ce deuxième texte est un extrait pris d'un article du journal anglais the Guardian.

In [15]:
text_2 = "In a landmark decision on one of three major climate cases, the first such rulings by an international court, the ECHR raised judicial pressure on governments to stop filling the atmosphere with gases that make extreme weather more violent."

classifier(text_2)

[{'label': 'HUMAN', 'score': 0.999920129776001}]

### Test 3

Ce troisème texte a été écrit par moi même.

In [16]:
text3 = "Traffic lights were invented by Garrett Morgan."

classifier(text3)

[{'label': 'HUMAN', 'score': 0.9986786246299744}]

In [18]:
result = classifier(text3)

label = result[0]['label']
score = result[0]['score']

print(f"Le troisième texte a été écrit par un {label.lower()}, avec une certitude de {round((score*100),2)}%")


Le troisième texte a été écrit par un human, avec une certitude de 99.87%


### Classification de données tirées de fichiers JSON

In [2]:
import json  # Importation de la bibliothèque json pour manipuler des fichiers JSON

# Chemins des fichiers d'entrée et de sortie
input_file = "test.jsonl"  # Chemin du fichier d'entrée au format JSONL
output_file = "data/toPredict_test.json"  # Chemin du fichier de sortie pour le nouveau fichier JSON

# Fonction pour extraire la réponse de chaque ligne du fichier JSONL
def extract_data_from_line(line):
    # Analyser le JSON de la ligne
    json_data = json.loads(line)
    # Extraire la réponse
    answer = json_data["Answer"]
    # Retourner la réponse
    return answer

# Fonction pour créer un nouveau fichier JSON avec uniquement les réponses
def create_new_json(input_file, output_file):
    new_data = []  # Liste pour contenir toutes les réponses
    with open(input_file, "r") as f:
        for line in f:
            # Extraction de la réponse de chaque ligne
            answer = extract_data_from_line(line)
            # Ajout de la réponse à new_data
            new_data.append({"text": answer})
    
    # Écriture de la liste new_data dans le fichier de sortie au format JSON
    with open(output_file, "w") as f:
        json.dump(new_data, f, indent=4)

# Appel de la fonction create_new_json avec les chemins des fichiers d'entrée et de sortie
create_new_json(input_file, output_file)


In [11]:
def classify(filename) :  # Définition de la fonction classify prenant en argument un nom de fichier

    # Instanciation d'un pipeline de classification de texte
    # en utilisant un modèle pré-entraîné nommé "model"
    classifier = pipeline("text-classification", model="model")  
    # Ouverture du fichier en mode lecture                                                             
    with open(filename, "r") as file:  
        # Chargement des données JSON à partir du fichier dans une variable nommée data
        data = json.load(file)  
    # Initialisation d'une liste vide pour stocker les nouvelles données classifiées
    new_data = []  

    # Parcours de chaque élément dans les données
    for element in data:
        # Extraction du texte de chaque élément
        text = element["text"]  
        
        # Vérification si la longueur du texte est inférieure ou égale à 512
        if len(text) <= 512:  
            # Classification du texte à l'aide du modèle de classification de texte
            result = classifier(text)  
            # Vérification du label prédit par le modèle
            if result[0]['label'] == "HUMAN":  
                # Attribution de l'étiquette 0 si le modèle prédit "HUMAN"
                label = 0  
            else :
                # Attribution de l'étiquette 1 si le modèle prédit autre chose
                label = 1  
            
            # Création d'un nouveau dictionnaire contenant le texte et l'étiquette
            new_dict = {'Answer' : text, 'label' : label}  
            # Ajout du nouveau dictionnaire à la liste new_data
            new_data.append(new_dict)  
        else : 
            # Si la longueur du texte dépasse 512 caractères, ne rien faire et passer à l'élément suivant
            # notre modèle a une limite de 512 tokens
            pass  
    # Retourne la liste des nouvelles données classifiées
    return new_data  

# Appel de la fonction classify avec le nom du fichier JSON en argument
classify("answers_test.json")  


[{'Answer': 'Life is a journey with a beginning and an end. We all embark on this journey at birth and most of us go on as long as we can. The funny thing about life is that it is a complex lottery, with ups and downs. Sadly we do not get to pick how and where we begin our lives, some are luckier than others but what we can do is try our best to live the life that we want for ourselves.',
  'label': 0},
 {'Answer': "I don't possess personal beliefs or emotions, but I can explore the concept from various perspectives. The meaning of life is a deeply philosophical question that has puzzled humanity for centuries. Some may find meaning through personal fulfillment, relationships, or contributing to the greater good. Others may find meaning in spiritual beliefs or the pursuit of knowledge and understanding. Ultimately, the meaning of life is subjective and can vary greatly from person to person.",
  'label': 1},
 {'Answer': 'If I could change one thing about the world, I would make sure th

Parmi ces 8 différents textes 7 sur 8 ont été correctement identifiés. Le dernier texte a été généré par ChatGPT mais est labelisé comme "humain". C'est probablement dû à la longueur du texte, un texte court est peut être plus difficile à classifier pour notre modèle. 

Faisons donc un nouveau test avec une ensemble de textes courts.

In [12]:
create_new_json("test2.jsonl", "data/toPredict_test2.json")
classify("answers_test2.json")

[{'Answer': 'Art is a free form of expression which can bring people together.',
  'label': 0},
 {'Answer': "Art has the power to inspire, provoke thought, evoke emotions, and promote cultural understanding, fostering creativity and expression while enriching society's cultural fabric.",
  'label': 0},
 {'Answer': 'Cultures are ideas, rules and traditions that link people to one another. It is the way one grows up and sometimes identifies with. Cultures can be points of reference which guide us through the journey of life.',
  'label': 0},
 {'Answer': 'Culture shapes our identities by influencing our beliefs, values, norms, behaviors, and worldview, providing a framework through which we understand ourselves and relate to others within society.',
  'label': 0}]

Nous constatons qu'avec des réponses plus courtes, il est difficile pour le modèle de bien identifier les textes de Chatgpt. Il a ici tout identifié comme textes écrits par un humain quand 2 textes sur 4 ont en réalité été généré par Chatgpt. 